### Data Scientist 

### Task and Data 

Using two input CSVs, create a Spark Scala/PySpark/Dask application (you can use any other distributed framework for data processing) which will process input data and create and store a new CSV with insights (what we call gleans).

The app with the code should be pushed to GitHub/GitLab or any other git based platform.

#### Input CSVs structure:

- invoice
 	invoice_id - Invoice UUID
	invoice_date - Issue date
	due_date - Due date
	period_start_date - Start date (if any) of service dates that invoice covers
	period_end_date - End date (if any) of service dates that invoice covers
	total_amount - Billed Amount
	canonical_vendor_id - Vendor UUID

- line_item
 	invoice_id - Invoice UUID
	line_item_id - Raw Line Item UUID
	period_start_date - Start date (if any) of service dates that raw line item covers
	period_end_date - End date (if any) of service dates that raw line item covers
	total_amount - Billed Amount
	canonical_line_item_id - Canonical line item UUID
    
    
#### Output CSV structure:

	glean_id - Glean UUID
	glean_date - Date when glean was triggered
	glean_text - Glean Text
	glean_type - Enum (vendor_not_seen_in_a_while, accrual_alert, large_month_increase_mtd, no_invoice_received)
	glean_location - Enum (invoice or vendor)
	invoice_id - Invoice UUID
	canonical_vendor_id - Vendor UUID    

#### Additional notes:
- 1:M relationship between invoice and line_item
- 1:M relationship between line_item_id and canonical_line_item_id (i.e. different raw line items can be mapped to a single canonical line item)
    


In [52]:
# Do all imports and installs here
import pandas as pd, re
import numpy as np 

import configparser
from datetime import datetime
import os

from pyspark.context import SparkContext
from pyspark.sql import SparkSession

import pyspark.sql.functions as func
import pyspark.sql.types as spark_types
from pyspark.sql.window import Window

from datetime import timedelta
from functools import reduce

pd.options.display.max_colwidth = 1000

**Note**: Set to path of data sources to process.

In [2]:
invoice_filename = "./invoice.csv"
line_item_filename = "./line_item.csv"

### Environment configuration

Below are example environment configuration options. Provided are examples for configuring AWS credentials or configuring a python for spark execution. Populate them if needed for the particular runtime environment.

In [3]:
#os.environ['AWS_ACCESS_KEY_ID']=''
#os.environ['AWS_SECRET_ACCESS_KEY']=''
#os.environ['PYSPARK_PYTHON'] = ''
#os.environ['PYSPARK_DRIVER_PYTHON'] = ''

In [4]:
# Create Spark Session for Data Process
spark = (
    SparkSession.builder
    .enableHiveSupport().getOrCreate()
)

#### Read data from csv 

In [5]:
invoice = spark.read.option("delimiter",",").option("header",True).csv(invoice_filename)

In [6]:
line_item = spark.read.option("delimiter",",").option("header",True).csv(line_item_filename)

### Gleans (insights) to be generated and backfilled in the past:

- glean_id - Glean UUID
- glean_date - Date when glean was triggered
- glean_text - Glean Text
- glean_type - Enum (vendor_not_seen_in_a_while, accrual_alert, large_month_increase_mtd, no_invoice_received)
- glean_location - Enum (invoice or vendor)
- invoice_id - Invoice UUID
- canonical_vendor_id - Vendor UUID  

In [7]:
class GleanOutput(object):
    def __init__(self, 
                 df, 
                 glean_text_format,
                 glean_text_args,
                 glean_type,
                 glean_location,
                 glean_date_column='invoice_date'
                ):
        
        self.glean_text_format = glean_text_format
        self.glean_text_args = glean_text_args
        self.glean_type = glean_type
        self.glean_location = glean_location
        self.glean_date_column = glean_date_column
        self.df = df

    def build_gleans(self):
        try:
            invoice_id = self.df.invoice_id
        except:
            invoice_id = (
                func.lit(None)
                .cast(spark_types.IntegerType())
                .alias('invoice_id')
            )
        gleans = (
            self.df.select(
                func.expr('uuid()').alias('glean_id'),
                self.df[self.glean_date_column].alias('glean_date'),
                func.format_string(
                    self.glean_text_format,
                    *self.glean_text_args
                ).alias("glean_text"),
                func.lit(self.glean_type).alias('glean_type'),
                func.lit(self.glean_location).alias('glean_location'), 
                invoice_id,
                self.df.canonical_vendor_id
             ) 
        )
        return gleans

### vendor_not_seen_in_a_while

	logic:
	1) don't trigger if invoice received from the vendor for the first time
	2) trigger if invoice received from the vendor and it's > 90 days since last `invoice_date`

	glean_text (text of the glean to be created):
		First new bill in [count_of_months_since_last_invoice] months from vendor [canonical_vendor_id]

	glean_location:
		invoice (this glean should be created on an invoice level)


In [8]:
window_spec = (
    Window
    .partitionBy(invoice.canonical_vendor_id)
    .orderBy(invoice.invoice_date)
)

invoice_with_lag = (
    invoice.withColumn(
        'lag_invoice_date',
        func.lag(invoice.invoice_date).over(window_spec)
    )
)

date_diff_expr = func.datediff(
    invoice_with_lag.invoice_date, 
    invoice_with_lag.lag_invoice_date
)

invoices_90_days = invoice_with_lag.filter(
    date_diff_expr > 90
) 

(
    invoices_90_days
    .withColumn('date_diff', date_diff_expr)
    .limit(5)
).toPandas()

,invoice_id,invoice_date,due_date,period_start_date,period_end_date,total_amount,canonical_vendor_id,lag_invoice_date,date_diff
0,e27451bb-b2eb-45e1-9b96-83c2f5399bcf,2020-01-20,2020-02-19,2019-12-20,2020-01-19,2833.08,c753ab1a-cd7b-4968-9a25-0f0471b0c882,2019-09-04,138
1,e0d0a045-5b38-4874-ab65-1bb521648dea,2020-09-21,2020-10-23,2020-09-21,2020-09-21,4850.39,0d41b0c0-dd27-4c32-909d-a190744e747b,2020-01-03,262
2,cc5cb581-1228-49c7-97a9-1af89a8423dc,2020-02-12,2020-03-13,2020-03-13,2020-03-13,2939.61,9be41a38-e810-48f6-8149-d9b19dee73e8,2019-06-11,246
3,0732547d-1e5c-4be3-8079-18e9a9fd02ef,2020-06-18,2020-07-18,2020-06-01,2020-08-31,13609.38,3a64161d-30c9-41b8-85e7-910b891e6520,2020-03-19,91
4,a205f7ca-d7ae-4c52-a6ac-8eb9e32a588f,2020-08-01,2020-08-31,2020-08-01,2021-07-31,16685.09,089685a6-ecec-44ab-bfdf-b2ccb5182382,2020-03-03,151


In [9]:
vendor_not_seen = GleanOutput(
    df=invoices_90_days,
    glean_text_format='First new bill in %d months from the vendor %s',
    glean_text_args=[
            func.months_between(
                invoices_90_days.invoice_date, 
                invoices_90_days.lag_invoice_date
            ).cast(spark_types.IntegerType()),
            invoices_90_days.canonical_vendor_id
    ],
    glean_location='invoice',
    glean_type='vendor_not_seen_in_a_while'
)
vendor_not_seen_df = vendor_not_seen.build_gleans()
vendor_not_seen_df.limit(5).toPandas()

#vendor_not_seen_in_a_while_gleans.limit(5).toPandas()

,glean_id,glean_date,glean_text,glean_type,glean_location,invoice_id,canonical_vendor_id
0,b797e910-c4ab-4563-9e2f-87388e97469c,2020-01-20,First new bill in 4 months from the vendor c753ab1a-cd7b-4968-9a25-0f0471b0c882,vendor_not_seen_in_a_while,invoice,e27451bb-b2eb-45e1-9b96-83c2f5399bcf,c753ab1a-cd7b-4968-9a25-0f0471b0c882
1,cf21b899-2b52-4984-b83a-192850c9a6c4,2020-09-21,First new bill in 8 months from the vendor 0d41b0c0-dd27-4c32-909d-a190744e747b,vendor_not_seen_in_a_while,invoice,e0d0a045-5b38-4874-ab65-1bb521648dea,0d41b0c0-dd27-4c32-909d-a190744e747b
2,075e3606-1fbf-4b5b-9972-8c90d9c2a911,2020-02-12,First new bill in 8 months from the vendor 9be41a38-e810-48f6-8149-d9b19dee73e8,vendor_not_seen_in_a_while,invoice,cc5cb581-1228-49c7-97a9-1af89a8423dc,9be41a38-e810-48f6-8149-d9b19dee73e8
3,33ccf443-dc97-4f31-8425-ff2535b30edb,2020-06-18,First new bill in 2 months from the vendor 3a64161d-30c9-41b8-85e7-910b891e6520,vendor_not_seen_in_a_while,invoice,0732547d-1e5c-4be3-8079-18e9a9fd02ef,3a64161d-30c9-41b8-85e7-910b891e6520
4,a630e8fc-fbbc-4b07-9580-9e11ff103c0e,2020-08-01,First new bill in 4 months from the vendor 089685a6-ecec-44ab-bfdf-b2ccb5182382,vendor_not_seen_in_a_while,invoice,a205f7ca-d7ae-4c52-a6ac-8eb9e32a588f,089685a6-ecec-44ab-bfdf-b2ccb5182382


### accrual_alert

	logic:
	1) trigger if `period_end_date` for invoice or any line item > 90 days after `invoice_date`
	   If there are multiple end dates, pick the last one.

	glean_text (text of the glean to be created):
		Line items from vendor [canonical_vendor_id] in this invoice cover future periods (through [period_end_date])

	glean_location:
		invoice (this glean should be created on an invoice level)

In [10]:
invoice_line_item = (
    invoice.join(line_item,
        invoice.invoice_id == line_item.invoice_id,
        how ='left'
            )
    .groupby(
        invoice.invoice_id,
        invoice.invoice_date, 
        invoice.canonical_vendor_id)
    .agg(func.max(line_item.period_end_date).alias('max_period_end_date')
        )
)

date_diff_expr = func.datediff(
    invoice_line_item.invoice_date, 
    invoice_line_item.max_period_end_date
)

invoices_90_days = invoice_line_item.filter(
    date_diff_expr > 90
)

(
    invoices_90_days
    .withColumn('date_diff', date_diff_expr)
    .limit(5)
).toPandas()

,invoice_id,invoice_date,canonical_vendor_id,max_period_end_date,date_diff
0,e1fbba74-6044-4a82-a571-871dcc05b1d1,2020-07-22,021143b8-0357-4836-ab87-50a9756d8156,2019-04-22,457
1,0669155f-41f4-478e-9728-f60ea0c0a12e,2020-01-31,a7f1156a-1898-40e9-8e72-52eb1f1471ca,2019-09-30,123
2,fc79cef0-d176-47f9-b9db-7153759c8019,2020-02-29,5bf79357-b95c-456c-bf8b-95d73a8a8f29,2018-10-30,487
3,816441c5-4c16-422c-a52d-254395d2e707,2019-11-19,a7f1156a-1898-40e9-8e72-52eb1f1471ca,2019-06-30,142
4,60ab208a-56dd-4dce-af12-e923a2c3b2cd,2020-01-01,906a0cbb-b05a-4501-beff-40a1d3bd6399,2014-10-05,1914


In [11]:
accural_alert = GleanOutput(
    df=invoices_90_days,
    glean_text_format='Line items from vendor %s in this invoice cover future periods (through %s)',
    glean_text_args=[
            invoices_90_days.canonical_vendor_id,
            invoices_90_days.max_period_end_date
    ],
    glean_location='invoice',
    glean_type='accrual_alert'
)
accural_alert_df = accural_alert.build_gleans()
accural_alert_df.limit(5).toPandas()


,glean_id,glean_date,glean_text,glean_type,glean_location,invoice_id,canonical_vendor_id
0,2f66ace6-e108-4622-a82f-7a5d47dc25d2,2020-07-22,Line items from vendor 021143b8-0357-4836-ab87-50a9756d8156 in this invoice cover future periods (through 2019-04-22),accrual_alert,invoice,e1fbba74-6044-4a82-a571-871dcc05b1d1,021143b8-0357-4836-ab87-50a9756d8156
1,4a2a4d8b-e1bc-4ac3-bdbc-292ad2f50e68,2020-01-31,Line items from vendor a7f1156a-1898-40e9-8e72-52eb1f1471ca in this invoice cover future periods (through 2019-09-30),accrual_alert,invoice,0669155f-41f4-478e-9728-f60ea0c0a12e,a7f1156a-1898-40e9-8e72-52eb1f1471ca
2,f1282d81-b098-4ce8-911c-d7dcff550bc9,2020-02-29,Line items from vendor 5bf79357-b95c-456c-bf8b-95d73a8a8f29 in this invoice cover future periods (through 2018-10-30),accrual_alert,invoice,fc79cef0-d176-47f9-b9db-7153759c8019,5bf79357-b95c-456c-bf8b-95d73a8a8f29
3,08e44bdc-3de0-46de-9d08-24a5731032f1,2019-11-19,Line items from vendor a7f1156a-1898-40e9-8e72-52eb1f1471ca in this invoice cover future periods (through 2019-06-30),accrual_alert,invoice,816441c5-4c16-422c-a52d-254395d2e707,a7f1156a-1898-40e9-8e72-52eb1f1471ca
4,28349eec-8045-4169-a42e-2baec0590de3,2020-01-01,Line items from vendor 906a0cbb-b05a-4501-beff-40a1d3bd6399 in this invoice cover future periods (through 2014-10-05),accrual_alert,invoice,60ab208a-56dd-4dce-af12-e923a2c3b2cd,906a0cbb-b05a-4501-beff-40a1d3bd6399


### large_month_increase_mtd

	logic:
	1) trigger if monthly spend > $10K and it increased > 50% of average spend over last 12 months. If monthly spend is less than $10K, > 200%. If less than $1K, > 500%. If less than $100, don't trigger the glean. Spend is sum of invoice `total_amount`.


	glean_text (text of the glean to be created):
		Monthly spend with [canonical_vendor_id] is $x (x%) higher than average

	glean_location:
		vendor (this glean should be created on a vendor level)


- glean_id - Glean UUID
- glean_date - Date when glean was triggered
- glean_text - Glean Text
- glean_type - Enum (vendor_not_seen_in_a_while, accrual_alert, large_month_increase_mtd, no_invoice_received)
- glean_location - Enum (invoice or vendor)
- invoice_id - Invoice UUID
- canonical_vendor_id - Vendor UUID  

In [27]:
monthly_total_df = (
    invoice.groupby(
        invoice.canonical_vendor_id,
        func.month(invoice.invoice_date).cast(spark_types.IntegerType()).alias("month"),
        func.year(invoice.invoice_date).cast(spark_types.IntegerType()).alias("year"), 
        (
            func.month(invoice.invoice_date).cast(spark_types.IntegerType()) 
            + func.year(invoice.invoice_date).cast(spark_types.IntegerType()) * 12)
        .alias('monthnum')
    )
    .agg(
        func.max(func.col('invoice_date')).alias('max_invoice_date'),
        func.sum(func.col('total_amount')).cast(spark_types.IntegerType()).alias('monthly_spend')
        )
    .sort(
        func.year(invoice.invoice_date).cast(spark_types.IntegerType()),
        func.month(invoice.invoice_date).cast(spark_types.IntegerType())
    )
)

monthly_total = monthly_total_df.alias("monthly_total")
preceding_months = monthly_total_df.alias('preceding_months')


monthly_spend_vs_prior_avg = (
    monthly_total.join(
        preceding_months,
        (
            (monthly_total.canonical_vendor_id == preceding_months.canonical_vendor_id) &
            (func.col("preceding_months.monthnum") >= func.col("monthly_total.monthnum") - 12) &
            (func.col("preceding_months.monthnum") <= func.col("monthly_total.monthnum") - 1 )
        )
        ,how ='left'
    )
    .groupby(
        monthly_total.monthly_spend,
        monthly_total.monthnum, 
        monthly_total.canonical_vendor_id, 
        monthly_total.max_invoice_date
    ).agg(
        func.avg(func.col('preceding_months.monthly_spend')).alias('TTM_avg'),
        func.count(func.col('preceding_months.monthly_spend')).alias('average_count')
    ).sort(
        monthly_total.canonical_vendor_id,
        monthly_total.monthnum
    )
)


monthly_spend_vs_prior_avg_filter = monthly_spend_vs_prior_avg.filter(
    ((monthly_spend_vs_prior_avg.monthly_spend >= 100) &
     (monthly_spend_vs_prior_avg.monthly_spend <= 1000) &
     ((monthly_spend_vs_prior_avg.monthly_spend / monthly_spend_vs_prior_avg.TTM_avg) > 6 )) |
    ((monthly_spend_vs_prior_avg.monthly_spend >= 1001) &
     (monthly_spend_vs_prior_avg.monthly_spend <= 10000) &
     ((monthly_spend_vs_prior_avg.monthly_spend / monthly_spend_vs_prior_avg.TTM_avg) > 3 )) |
    ((monthly_spend_vs_prior_avg.monthly_spend >= 100001) &
     ((monthly_spend_vs_prior_avg.monthly_spend / monthly_spend_vs_prior_avg.TTM_avg) > 1.5 ) 
)
)

In [28]:
mtd_increase = GleanOutput(
    df=monthly_spend_vs_prior_avg_filter,
    glean_text_format='Monthly spend with %s is %s higher than average',
    glean_text_args=[
        monthly_spend_vs_prior_avg_filter.canonical_vendor_id, 
        (monthly_spend_vs_prior_avg_filter.monthly_spend / monthly_spend_vs_prior_avg_filter.TTM_avg)
    ],
    glean_location='vendor',
    glean_type = 'large_month_increase_mtd',
    glean_date_column = 'max_invoice_date'
)
mtd_increase_df = mtd_increase.build_gleans().cache()
mtd_increase_df.limit(5).toPandas()


,glean_id,glean_date,glean_text,glean_type,glean_location,invoice_id,canonical_vendor_id
0,fd682898-4800-4caa-a781-0c0d6aa456fa,2020-09-21,Monthly spend with 0d41b0c0-dd27-4c32-909d-a190744e747b is 3.4348441926345608 higher than average,large_month_increase_mtd,vendor,NaN,0d41b0c0-dd27-4c32-909d-a190744e747b
1,5ee348cd-0396-4d1c-ab0a-18bb81aedbd0,2020-07-21,Monthly spend with 0fe5f1a4-849a-4e7c-bb11-00e5e9d6cec0 is 5.245235361653272 higher than average,large_month_increase_mtd,vendor,NaN,0fe5f1a4-849a-4e7c-bb11-00e5e9d6cec0
2,d26d697e-aec4-4a68-b1a0-9ac6223adeb5,2020-10-13,Monthly spend with 0fe5f1a4-849a-4e7c-bb11-00e5e9d6cec0 is 1.9046202932707808 higher than average,large_month_increase_mtd,vendor,NaN,0fe5f1a4-849a-4e7c-bb11-00e5e9d6cec0
3,4e2afaf2-6968-4b62-bb3c-a2f9a739fa5f,2020-05-18,Monthly spend with 10eaea15-03c3-4d52-9120-9fa817ae4fd3 is 3.2503445107946716 higher than average,large_month_increase_mtd,vendor,NaN,10eaea15-03c3-4d52-9120-9fa817ae4fd3
4,697ea613-bd69-4811-bc1e-625c43149636,2020-06-30,Monthly spend with 122c91c1-d193-4e6e-aae4-4c480aafec6e is 31.844638949671772 higher than average,large_month_increase_mtd,vendor,NaN,122c91c1-d193-4e6e-aae4-4c480aafec6e


### no_invoice_received

	logic:
	1) trigger if vendor sends invoice(s) either on MONTHLY basis (1 or more per month) or QUARTERLY basis (1 per quarter).

		1.1) MONTHLY case: trigger if there were 3 consecutive months with invoices received from vendor but there are no invoices received in current month.
			 Start triggering the glean from the day when vendor usually sends the invoice (you need to count day frequency). Keep triggering the glean till the end of the current month or until the day when new invoice received.
			 If there are multiple days based on frequency count, pick the earliest one.

		1.2) QUARTERLY case: trigger if there were 2 consecutive quarters with invoices received from vendor but there are no invoices received in current quarter.
			 Start triggering the glean from the day when vendor usually sends the invoice (you need to count day frequency). Keep triggering the glean till the end of the current month of the quarter or until the day when new invoice received.
			 If there are multiple days based on frequency count, pick the earliest one.


	glean_text (text of the glean to be created):
		[canonical_vendor_id] generally charges between on [most_frequent_day_number] day of each month invoices are sent. On [date], an invoice from [canonical_vendor_id] has not been received

	glean_location:
		vendor (this glean should be created on a vendor level)

In [29]:
# Relevant_date_range for create date dimension table

date_range_results = (
    invoice.
    agg(
        func.min(invoice.invoice_date).alias('min_date'),
        func.max(invoice.invoice_date).alias('max_date')        
        )
)

min_date, max_date = date_range_results.take(1)[0]

end_of_next_quarter = (
    pd.to_datetime(max_date) 
    + pd.tseries.offsets.DateOffset(months=3) 
    + pd.tseries.offsets.QuarterEnd()
)
    
relevant_date_range = pd.date_range(min_date, end_of_next_quarter)
    
    
day_of_quarter = (
    relevant_date_range - (relevant_date_range - pd.tseries.offsets.QuarterEnd())
).days

In [30]:
# Create date dimension table
pd_all_days = pd.DataFrame(
    {
        "date":relevant_date_range.date, 
        'day_of_quarter':day_of_quarter,
        'month_number':relevant_date_range.month + relevant_date_range.year * 12,
        'day_of_month': relevant_date_range.day,
        'quarter_number': (relevant_date_range.month - 1) // 3 + relevant_date_range.year * 4
    }
)

date_dimension = spark.createDataFrame(pd_all_days)
date_dimension.registerTempTable("date_dimension")

In [31]:
# day_of_month_frequencies
dom_frequencies = (
    invoice.filter(invoice.invoice_date.isNotNull())
    .groupBy(
        invoice.canonical_vendor_id,
        func.dayofmonth(invoice.invoice_date).cast(spark_types.IntegerType()).alias('day_of_month')      
    ).agg(func.count(func.lit(1)).alias('frequency'))
    .sort(
        invoice.canonical_vendor_id,
        func.dayofmonth(invoice.invoice_date).cast(spark_types.IntegerType())
    )
)


In [32]:
# day_of_quarter_frequencies

doq_frequencies = (
    invoice.join(date_dimension,
                invoice.invoice_date == date_dimension.date
                ).groupBy(
        invoice.canonical_vendor_id,
        date_dimension.day_of_quarter
    ).agg(func.count(func.lit(1)).alias('frequency'))
    .sort(
        invoice.canonical_vendor_id,
        date_dimension.day_of_quarter
    )
)


In [33]:
# most_frequent_day
window_spec = (
    Window
    .partitionBy(dom_frequencies.canonical_vendor_id)
    .orderBy(
        dom_frequencies.frequency.desc(),
        dom_frequencies.day_of_month
    )
)

most_frequent_day = (
    dom_frequencies.select(
        dom_frequencies.canonical_vendor_id,
        func.first(dom_frequencies.day_of_month).over(window_spec).alias('most_frequent_day')
    ).orderBy(dom_frequencies.canonical_vendor_id).distinct()
)


In [34]:
# most_frequent_day_quarter
window_spec = (
    Window
    .partitionBy(doq_frequencies.canonical_vendor_id)
    .orderBy(
        doq_frequencies.frequency.desc(),
        doq_frequencies.day_of_quarter
    )
)

most_frequent_day_quarter = (
    doq_frequencies.select(
        doq_frequencies.canonical_vendor_id,
        func.first(doq_frequencies.day_of_quarter).over(window_spec).alias('most_frequent_day_quarter')
    ).orderBy(dom_frequencies.canonical_vendor_id).distinct()
    
)


In [36]:
#invoice_month_count
invoice_with_month_number = ( 
    invoice.withColumn(
    'month_number',
    func.year(invoice.invoice_date).cast(spark_types.IntegerType()) * 12 + 
    func.month(invoice.invoice_date).cast(spark_types.IntegerType())
)
)


invoice_month_counts = (
    most_frequent_day.join(
        date_dimension,
        date_dimension.day_of_month > most_frequent_day.most_frequent_day
    )
    .join(invoice_with_month_number, 
             (most_frequent_day.canonical_vendor_id == invoice_with_month_number.canonical_vendor_id)
            & (func.col("invoice_date") <= date_dimension.date)
    )
    .groupBy(
        date_dimension.date, 
        date_dimension.day_of_month, 
        most_frequent_day.canonical_vendor_id,
        most_frequent_day.most_frequent_day)
    .agg(
        func.countDistinct(
            func.when(
                invoice_with_month_number.month_number == date_dimension.month_number,
                invoice_with_month_number.month_number
            )
        ).alias('invoice_count_for_current_month'),
        func.countDistinct(
            func.when(          
                (invoice_with_month_number.month_number >= date_dimension.month_number - 3)
              & (invoice_with_month_number.month_number <= date_dimension.month_number - 1)
                ,
                invoice_with_month_number.month_number
            )
        ).alias('invoice_count_for_preceding_month')
        
    )
)

In [37]:
(most_frequent_day.join(
        date_dimension,
        date_dimension.day_of_month > most_frequent_day.most_frequent_day
    )
    .join(invoice_with_month_number, 
             (most_frequent_day.canonical_vendor_id == invoice_with_month_number.canonical_vendor_id)
            & (func.col("invoice_date") <= date_dimension.date)
    )
    .groupBy(
        date_dimension.date, 
        date_dimension.day_of_month, 
        most_frequent_day.canonical_vendor_id,
        most_frequent_day.most_frequent_day)
    .agg(
        ((func.countDistinct(
            func.when(
                invoice_with_month_number.month_number == date_dimension.month_number,
                invoice_with_month_number.month_number
            )
        ) == 0) &
        (func.countDistinct(
            func.when(          
                (invoice_with_month_number.month_number >= date_dimension.month_number - 3)
              & (invoice_with_month_number.month_number <= date_dimension.month_number - 1)
                ,
                invoice_with_month_number.month_number
            )
        ) == 3)).alias('glean')
    )   
).show()

+----------+------------+--------------------+-----------------+-----+
|      date|day_of_month| canonical_vendor_id|most_frequent_day|glean|
+----------+------------+--------------------+-----------------+-----+
|2020-12-17|          17|7409fa50-fde9-493...|                3|false|
|2021-03-26|          26|bd81a118-4b34-4c0...|               25|false|
|2019-10-22|          22|a77018fe-ac76-45b...|                2|false|
|2020-09-12|          12|c753ab1a-cd7b-496...|                4| true|
|2020-11-21|          21|39d309ff-4df2-4b8...|                5|false|
|2020-12-14|          14|3629f0a6-158e-402...|                1|false|
|2019-10-22|          22|3aae2c5f-13a4-438...|               15|false|
|2020-07-13|          13|88c2f882-10f4-4e9...|                7|false|
|2020-07-27|          27|10eaea15-03c3-4d5...|               17|false|
|2020-10-17|          17|906a0cbb-b05a-450...|                1| true|
|2020-10-24|          24|37a1712d-f876-428...|               15|false|
|2019-

In [43]:
(invoice_month_counts
    .filter(invoice_month_counts.invoice_count_for_current_month == 0)
    .filter(invoice_month_counts.invoice_count_for_preceding_month == 3)
).limit(4).toPandas()

,date,day_of_month,canonical_vendor_id,most_frequent_day,invoice_count_for_current_month,invoice_count_for_preceding_month
0,2020-09-12,12,c753ab1a-cd7b-4968-9a25-0f0471b0c882,4,0,3
1,2020-10-17,17,906a0cbb-b05a-4501-beff-40a1d3bd6399,1,0,3
2,2020-05-27,27,1e71abd5-41e3-488d-bb1c-9e48f3790846,22,0,3
3,2020-05-18,18,335fd6c2-e48b-4a71-96c3-71efae5ffe30,1,0,3


In [39]:
#invoice_quarter_count

invoice_with_quarter_number =(
    invoice.withColumn(
        'quarter_number',
        func.year(invoice.invoice_date).cast(spark_types.IntegerType()) * 4 + 
        func.floor(func.month(invoice.invoice_date).cast(spark_types.IntegerType())/3)
    )
)
#invoice_with_quarter_number.show()


In [40]:
invoice_quarter_counts = (
    most_frequent_day_quarter.join(
        date_dimension,
        date_dimension.day_of_quarter > most_frequent_day_quarter.most_frequent_day_quarter
    )
    .join(invoice_with_quarter_number, 
             (most_frequent_day_quarter.canonical_vendor_id == invoice_with_quarter_number.canonical_vendor_id)
            & (func.col("invoice_date") <= date_dimension.date)
    )
    .groupBy(
        date_dimension.date, 
        date_dimension.day_of_quarter, 
        most_frequent_day_quarter.canonical_vendor_id,
        most_frequent_day_quarter.most_frequent_day_quarter)
    .agg(
        func.count(
            func.when(
                invoice_with_quarter_number.quarter_number == date_dimension.quarter_number,
                invoice_with_quarter_number.quarter_number
            )
        ).alias('invoice_count_for_current_quarter'),
        func.count(
            func.when(
                invoice_with_quarter_number.quarter_number == date_dimension.quarter_number - 1,
                invoice_with_quarter_number.quarter_number
            )
        ).alias('invoice_count_for_preceding_quarter_1'),
        func.count(
            func.when(
                invoice_with_quarter_number.quarter_number == date_dimension.quarter_number - 2,
                invoice_with_quarter_number.quarter_number
            )
        ).alias('invoice_count_for_preceding_quarter_2')
        
    )
)

In [41]:
#no_invoice_received_month

no_invoice_received_month_filter = (
    invoice_month_counts
    .filter(invoice_month_counts.invoice_count_for_current_month == 0)
    .filter(invoice_month_counts.invoice_count_for_preceding_month == 3)
)    


no_invoice_received_month = GleanOutput(
    df=no_invoice_received_month_filter,
    glean_text_format='%s generally charges between on %s day of each month invoices are sent. On %s, an invoice from %s has not been received',
    glean_text_args=[
            invoice_month_counts.canonical_vendor_id,
            most_frequent_day.most_frequent_day,
            invoice_month_counts.date,
            invoice_month_counts.canonical_vendor_id,
    ],
    glean_date_column='date',
    glean_location='vendor',
    glean_type='no_invoice_received'
)
no_invoice_received_month_df = no_invoice_received_month.build_gleans()
no_invoice_received_month_df.limit(5).toPandas()

,glean_id,glean_date,glean_text,glean_type,glean_location,invoice_id,canonical_vendor_id
0,28c62e0e-7f4a-4067-b12d-903699bd0ccc,2020-09-12,"c753ab1a-cd7b-4968-9a25-0f0471b0c882 generally charges between on 4 day of each month invoices are sent. On 18517, an invoice from c753ab1a-cd7b-4968-9a25-0f0471b0c882 has not been received",no_invoice_received,vendor,NaN,c753ab1a-cd7b-4968-9a25-0f0471b0c882
1,fabd8f53-b257-43c5-b154-76198b826c90,2020-10-17,"906a0cbb-b05a-4501-beff-40a1d3bd6399 generally charges between on 1 day of each month invoices are sent. On 18552, an invoice from 906a0cbb-b05a-4501-beff-40a1d3bd6399 has not been received",no_invoice_received,vendor,NaN,906a0cbb-b05a-4501-beff-40a1d3bd6399
2,300ace32-a04a-4782-a33d-a2c5608586bd,2020-05-27,"1e71abd5-41e3-488d-bb1c-9e48f3790846 generally charges between on 22 day of each month invoices are sent. On 18409, an invoice from 1e71abd5-41e3-488d-bb1c-9e48f3790846 has not been received",no_invoice_received,vendor,NaN,1e71abd5-41e3-488d-bb1c-9e48f3790846
3,2b70b429-e2c6-402f-bbb8-c44035049932,2020-05-18,"335fd6c2-e48b-4a71-96c3-71efae5ffe30 generally charges between on 1 day of each month invoices are sent. On 18400, an invoice from 335fd6c2-e48b-4a71-96c3-71efae5ffe30 has not been received",no_invoice_received,vendor,NaN,335fd6c2-e48b-4a71-96c3-71efae5ffe30
4,dce1f680-3bba-4795-8e5f-1d5542d52f7e,2020-08-30,"b21a0540-aaff-4d8d-ad8e-4430809fe0ff generally charges between on 15 day of each month invoices are sent. On 18504, an invoice from b21a0540-aaff-4d8d-ad8e-4430809fe0ff has not been received",no_invoice_received,vendor,NaN,b21a0540-aaff-4d8d-ad8e-4430809fe0ff


In [42]:
#no_invoice_received_quarter

no_invoice_received_quarter_filter = (
    invoice_quarter_counts
    .filter(invoice_quarter_counts.invoice_count_for_current_quarter == 0)
    .filter(invoice_quarter_counts.invoice_count_for_preceding_quarter_1 == 1)
    .filter(invoice_quarter_counts.invoice_count_for_preceding_quarter_2 == 1)
)    


no_invoice_received_quarter = GleanOutput(
    df=no_invoice_received_quarter_filter,
    glean_text_format='%s generally charges between on %s day of each month invoices are sent. On %s, an invoice from %s has not been received',
    glean_text_args=[
            invoice_quarter_counts.canonical_vendor_id,
            most_frequent_day_quarter.most_frequent_day_quarter,
            func.date_format(
                no_invoice_received_quarter_filter.date,
                'yyyy-M-dd'
            ),
            invoice_quarter_counts.canonical_vendor_id
    ],
    glean_date_column='date',
    glean_location='vendor',
    glean_type='no_invoice_received'
)
no_invoice_received_quarter_df = no_invoice_received_quarter.build_gleans()
no_invoice_received_quarter_df.limit(5).toPandas()


,glean_id,glean_date,glean_text,glean_type,glean_location,invoice_id,canonical_vendor_id
0,46eff38b-d6a3-484c-a7e1-7b736c22b210,2020-11-19,"8db7096a-752d-433f-a263-555b035c7836 generally charges between on 24 day of each month invoices are sent. On 2020-11-19, an invoice from 8db7096a-752d-433f-a263-555b035c7836 has not been received",no_invoice_received,vendor,NaN,8db7096a-752d-433f-a263-555b035c7836
1,3f2935c2-4776-4b4d-bbc1-eeb62b69dcca,2020-10-31,"88841f65-a1b4-4daa-ad7c-50140bb329cb generally charges between on 22 day of each month invoices are sent. On 2020-10-31, an invoice from 88841f65-a1b4-4daa-ad7c-50140bb329cb has not been received",no_invoice_received,vendor,NaN,88841f65-a1b4-4daa-ad7c-50140bb329cb
2,930c1b5c-633d-4e99-954c-b6c0058e3449,2020-09-26,"aee7d766-492f-42cb-b5c0-44744c2d8592 generally charges between on 15 day of each month invoices are sent. On 2020-9-26, an invoice from aee7d766-492f-42cb-b5c0-44744c2d8592 has not been received",no_invoice_received,vendor,NaN,aee7d766-492f-42cb-b5c0-44744c2d8592
3,860a621b-cc00-44ff-9d09-1c70de058d0c,2020-09-16,"9b4e7969-6072-4003-84c0-9c69af613fbe generally charges between on 41 day of each month invoices are sent. On 2020-9-16, an invoice from 9b4e7969-6072-4003-84c0-9c69af613fbe has not been received",no_invoice_received,vendor,NaN,9b4e7969-6072-4003-84c0-9c69af613fbe
4,d1d4fdc2-62b0-435a-98de-7527823ac9b5,2020-10-29,"8f3ae183-3da9-4113-b432-e559dcba8f1c generally charges between on 17 day of each month invoices are sent. On 2020-10-29, an invoice from 8f3ae183-3da9-4113-b432-e559dcba8f1c has not been received",no_invoice_received,vendor,NaN,8f3ae183-3da9-4113-b432-e559dcba8f1c


In [ ]:
spark.sql("select * from no_invoice_received_quarter").limit(1).toPandas().T

### Union all glean data 

In [53]:
def union_dataframes(dfs):
    return reduce(
        lambda df1, df2: df1.union(df2), 
        dfs
    )

In [54]:
combined_gleans = union_dataframes([
    accural_alert_df,
    vendor_not_seen_df,
    mtd_increase_df,
    no_invoice_received_month_df,
    no_invoice_received_quarter_df
])

In [55]:
combined_gleans.count()

3306

In [56]:
combined_gleans.head()

Row(glean_id='2f66ace6-e108-4622-a82f-7a5d47dc25d2', glean_date='2020-07-22', glean_text='Line items from vendor 021143b8-0357-4836-ab87-50a9756d8156 in this invoice cover future periods (through 2019-04-22)', glean_type='accrual_alert', glean_location='invoice', invoice_id='e1fbba74-6044-4a82-a571-871dcc05b1d1', canonical_vendor_id='021143b8-0357-4836-ab87-50a9756d8156')

In [59]:
combined_gleans.repartition(1).write.format('com.databricks.spark.csv').save('combined_gleans_v1',header = True)